In [1]:
import numpy as np
import cv2

In [2]:
# 動画の読み込み
cap = cv2.VideoCapture('../input/movie/raw/timelapse.mp4')

KERNEL_SIZE = 15

# config
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(
    '../output/movie/output.mp4',
    fourcc,
    fps,
    (width, height),
    isColor = True
)


while True:
    ret, frame = cap.read()

    if not ret:
        break

    # gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # binarization using Otsu's method
    _, th = cv2.threshold(frame_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # configure the kernel
    kernel = np.ones((KERNEL_SIZE, KERNEL_SIZE),np.uint8)

    # morphological transformation(Dilation)
    th_dilation = cv2.dilate(th, kernel, iterations = 1)
    
    # contour extraction
    contours, hierarchy = cv2.findContours(th_dilation,
                                           cv2.RETR_LIST,
                                           cv2.CHAIN_APPROX_NONE)

    # draw the contours on the source image
    img_contour = cv2.drawContours(frame.copy(), contours, -1, (0, 255, 0), 2)

    cv2.imshow('frame', img_contour)

    writer.write(img_contour)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break


cap.release()
writer.release()
cv2.destroyAllWindows()
